In [56]:
import pandas as pd

In [57]:
df = pd.read_csv('../data/movielens.csv')

In [58]:
df.head()

,bucketized_user_age,movie_genres,movie_id,movie_title,raw_user_age,timestamp,user_gender,user_id,user_occupation_label,user_occupation_text,user_rating,user_zip_code
0,45.0,[7],b'357',"b""One Flew Over the Cuckoo's Nest (1975)""",46.0,879024327,True,b'138',4,b'doctor',4.0,b'53211'
1,25.0,[ 4 14],b'709',b'Strictly Ballroom (1992)',32.0,875654590,True,b'92',5,b'entertainment',2.0,b'80525'
2,18.0,[4],b'412',"b'Very Brady Sequel, A (1996)'",24.0,882075110,True,b'301',17,b'student',4.0,b'55439'
3,50.0,[5 7],b'56',b'Pulp Fiction (1994)',50.0,883326919,True,b'60',4,b'healthcare',4.0,b'06472'
4,50.0,[10 16],b'895',b'Scream 2 (1997)',55.0,891409199,True,b'197',18,b'technician',3.0,b'75094'


In [60]:
df[df['user_id'] == '138']

,bucketized_user_age,movie_genres,movie_id,movie_title,raw_user_age,timestamp,user_gender,user_id,user_occupation_label,user_occupation_text,user_rating,user_zip_code


In [12]:
def trim_col(x):
    return x[2:-1]

df['movie_title'] = df['movie_title'].apply(trim_col)
df['movie_id'] = df['movie_id'].apply(trim_col)
df['user_id'] = df['user_id'].apply(trim_col)
df['user_occupation_text'] = df['user_occupation_text'].apply(trim_col)
df['user_zip_code'] = df['user_zip_code'].apply(trim_col)

In [13]:
def genre(x):
    x = x[1:-1]
    x = x.strip()
    x = x.split(sep=' ')
    l = list(filter(lambda x: x not in (None, ""), x))

    return l

In [14]:
df['movie_genres'] = df['movie_genres'].apply(genre)

In [15]:
df.head()

,bucketized_user_age,movie_genres,movie_id,movie_title,raw_user_age,timestamp,user_gender,user_id,user_occupation_label,user_occupation_text,user_rating,user_zip_code
0,45.0,[7],357,One Flew Over the Cuckoo's Nest (1975),46.0,879024327,True,138,4,doctor,4.0,53211
1,25.0,"[4, 14]",709,Strictly Ballroom (1992),32.0,875654590,True,92,5,entertainment,2.0,80525
2,18.0,[4],412,"Very Brady Sequel, A (1996)",24.0,882075110,True,301,17,student,4.0,55439
3,50.0,"[5, 7]",56,Pulp Fiction (1994),50.0,883326919,True,60,4,healthcare,4.0,06472
4,50.0,"[10, 16]",895,Scream 2 (1997),55.0,891409199,True,197,18,technician,3.0,75094


In [16]:
genre_map = {
    0: 'Unknown',
    1: 'Action',
    2: 'Adventure',
    3: 'Animation',
    4: 'Children',
    5: 'Comedy',
    6: 'Crime',
    7: 'Documentary',
    8: 'Drama',
    9: 'Fantasy',
    10: 'Film-Noir',
    11: 'Horror',
    12: 'Musical',
    13: 'Mystery',
    14: 'Romance',
    15: 'Sci-Fi',
    16: 'Thriller',
    17: 'War',
    18: 'Western',
    19: 'Other'
}

In [9]:
df["movie_genres"] = df["movie_genres"].apply(lambda ids: [genre_map[int(id)] for id in ids])

In [10]:
df.head()

,bucketized_user_age,movie_genres,movie_id,movie_title,raw_user_age,timestamp,user_gender,user_id,user_occupation_label,user_occupation_text,user_rating,user_zip_code
0,45.0,[Documentary],357,One Flew Over the Cuckoo's Nest (1975),46.0,879024327,True,138,4,doctor,4.0,53211
1,25.0,"[Children, Romance]",709,Strictly Ballroom (1992),32.0,875654590,True,92,5,entertainment,2.0,80525
2,18.0,[Children],412,"Very Brady Sequel, A (1996)",24.0,882075110,True,301,17,student,4.0,55439
3,50.0,"[Comedy, Documentary]",56,Pulp Fiction (1994),50.0,883326919,True,60,4,healthcare,4.0,06472
4,50.0,"[Film-Noir, Thriller]",895,Scream 2 (1997),55.0,891409199,True,197,18,technician,3.0,75094


In [11]:
df['user_gender'].value_counts()

user_gender
True     74260
False    25740
Name: count, dtype: int64

In [12]:
df.dtypes

bucketized_user_age      float64
movie_genres              object
movie_id                  object
movie_title               object
raw_user_age             float64
timestamp                  int64
user_gender                 bool
user_id                   object
user_occupation_label      int64
user_occupation_text      object
user_rating              float64
user_zip_code             object
dtype: object

In [13]:
df.isna().sum()

bucketized_user_age      0
movie_genres             0
movie_id                 0
movie_title              0
raw_user_age             0
timestamp                0
user_gender              0
user_id                  0
user_occupation_label    0
user_occupation_text     0
user_rating              0
user_zip_code            0
dtype: int64

In [14]:
from sklearn.preprocessing import MultiLabelBinarizer

In [15]:
mlb = MultiLabelBinarizer()

In [16]:
genres_encoded = mlb.fit_transform(df['movie_genres'])
genres_df = pd.DataFrame(genres_encoded, columns=mlb.classes_)

In [17]:
df = pd.concat([df, genres_df], axis=1).drop('movie_genres', axis=1)

In [18]:
df.head()

,bucketized_user_age,movie_id,movie_title,raw_user_age,timestamp,user_gender,user_id,user_occupation_label,user_occupation_text,user_rating,...,Film-Noir,Musical,Mystery,Other,Romance,Sci-Fi,Thriller,Unknown,War,Western
0,45.0,357,One Flew Over the Cuckoo's Nest (1975),46.0,879024327,True,138,4,doctor,4.0,...,0,0,0,0,0,0,0,0,0,0
1,25.0,709,Strictly Ballroom (1992),32.0,875654590,True,92,5,entertainment,2.0,...,0,0,0,0,1,0,0,0,0,0
2,18.0,412,"Very Brady Sequel, A (1996)",24.0,882075110,True,301,17,student,4.0,...,0,0,0,0,0,0,0,0,0,0
3,50.0,56,Pulp Fiction (1994),50.0,883326919,True,60,4,healthcare,4.0,...,0,0,0,0,0,0,0,0,0,0
4,50.0,895,Scream 2 (1997),55.0,891409199,True,197,18,technician,3.0,...,1,0,0,0,0,0,1,0,0,0


In [19]:
from sentence_transformers import SentenceTransformer

In [20]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [21]:
# This line of code kept running more than 20min so we moved to another optimezed solution in the next cell.

# df['title_embedding'] = df['movie_title'].apply(lambda title: model.encode(title))

In [22]:
embeddings = model.encode(df['movie_title'].tolist(), convert_to_tensor=True)
embeddings.shape

torch.Size([100000, 384])

In [23]:
embeddings

tensor([[-0.0351,  0.0105, -0.0312,  ..., -0.0218, -0.0094,  0.0348],
        [ 0.0086, -0.0072, -0.0644,  ...,  0.0253,  0.0306, -0.0308],
        [-0.0321, -0.0142, -0.0637,  ...,  0.0862,  0.0505,  0.0244],
        ...,
        [-0.0351, -0.0501, -0.0722,  ..., -0.0219, -0.0168,  0.0162],
        [-0.0337,  0.0383, -0.0397,  ...,  0.0653, -0.0048,  0.0151],
        [-0.0404,  0.0470, -0.0209,  ...,  0.1131, -0.0211, -0.0109]])

In [24]:
embeddings_list = embeddings.tolist()
df['title_embedding'] = embeddings_list

In [25]:
embeddings_df = pd.DataFrame(df['title_embedding'].tolist(), index=df.index)

In [26]:
df = pd.concat([df.drop(columns=['title_embedding']), embeddings_df], axis=1)

In [27]:
df.head()

,bucketized_user_age,movie_id,movie_title,raw_user_age,timestamp,user_gender,user_id,user_occupation_label,user_occupation_text,user_rating,...,374,375,376,377,378,379,380,381,382,383
0,45.0,357,One Flew Over the Cuckoo's Nest (1975),46.0,879024327,True,138,4,doctor,4.0,...,-0.000059,0.028617,-0.006242,-0.010861,-0.026007,-0.070109,0.077629,-0.021817,-0.009381,0.034754
1,25.0,709,Strictly Ballroom (1992),32.0,875654590,True,92,5,entertainment,2.0,...,-0.020182,-0.000871,-0.000600,0.060614,-0.050331,0.027037,0.017307,0.025332,0.030639,-0.030832
2,18.0,412,"Very Brady Sequel, A (1996)",24.0,882075110,True,301,17,student,4.0,...,0.028449,0.011174,-0.009052,0.007225,-0.005080,0.015241,0.099558,0.086217,0.050451,0.024373
3,50.0,56,Pulp Fiction (1994),50.0,883326919,True,60,4,healthcare,4.0,...,0.022719,0.018235,0.011511,0.137771,-0.051351,0.016533,0.010268,-0.001654,0.101332,0.017574
4,50.0,895,Scream 2 (1997),55.0,891409199,True,197,18,technician,3.0,...,0.068550,0.025233,-0.034744,0.048030,-0.023011,0.035936,0.086280,-0.033038,0.083152,0.024514


In [28]:
# synapses
# NMF

In [37]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, MultiLabelBinarizer
from sentence_transformers import SentenceTransformer

# Load the dataset
# data = pd.read_csv("../data/movielens.csv")
data = df

# Initialize transformers
# genre_encoder = OneHotEncoder(sparse=False)
genre_encoder = MultiLabelBinarizer()
occupation_encoder = OneHotEncoder(sparse=False)
scaler = StandardScaler()
title_transformer = SentenceTransformer('all-MiniLM-L6-v2')

genre_map = {
    0: 'Unknown',
    1: 'Action',
    2: 'Adventure',
    3: 'Animation',
    4: 'Children',
    5: 'Comedy',
    6: 'Crime',
    7: 'Documentary',
    8: 'Drama',
    9: 'Fantasy',
    10: 'Film-Noir',
    11: 'Horror',
    12: 'Musical',
    13: 'Mystery',
    14: 'Romance',
    15: 'Sci-Fi',
    16: 'Thriller',
    17: 'War',
    18: 'Western',
    19: 'Other'
}


# Process user features
def preprocess_user_features(data):
    # Normalize bucketized_user_age
    bucketized_user_age = scaler.fit_transform(data[['bucketized_user_age']])
    
    # Encode user_gender as binary (0: False, 1: True)
    user_gender = data['user_gender'].astype(int).values.reshape(-1, 1)
    
    # One-hot encode user_occupation_label
    occupation_encoded = occupation_encoder.fit_transform(data[['user_occupation_text']])
    
    # Optional: Encode user_zip_code (use first 3 digits as region)
    # data['zip_region'] = data['user_zip_code'].astype(str).str[:3]
    # zip_encoded = LabelEncoder().fit_transform(data['zip_region']).reshape(-1, 1)
    
    # Return user feature vector
    user_features = np.hstack([bucketized_user_age, user_gender, occupation_encoded]) #, zip_encoded])
    return user_features

# Process movie features
def preprocess_movie_features(data):
    """
    Preprocess movie features for model input.

    Args:
        data (pd.DataFrame): DataFrame containing movie features, including 'movie_genres' and 'movie_title'.
        title_transformer: Sentence transformer model for title embeddings.
        genre_encoder: MultiLabelBinarizer for encoding genres (optional; will be fitted if not provided).

    Returns:
        np.ndarray: Preprocessed movie feature vectors.
    """
    # One-hot encode movie_genres (assuming genres are in a list format)
    # genres = data['movie_genres'].apply(lambda x: [int(genre) for genre in x.strip('[]').split()])
    # genre_encoded = genre_encoder.fit_transform(genres.tolist())
    data["movie_genres"] = data["movie_genres"].apply(
        lambda ids: [genre_map[int(id)] for id in ids.strip('[]').split()]
    )

    # MultiLabelBinarizer for genres
    genre_encoded = genre_encoder.fit_transform(data["movie_genres"])
    
    # Encode movie titles with sentence embeddings
    title_embeddings = np.array(title_transformer.encode(data['movie_title'].tolist()))
    
    # Map movie_id to embeddings (if using embedding layer)
    # movie_ids = LabelEncoder().fit_transform(data['movie_id'])
    # movie_id_embeddings = np.expand_dims(movie_ids, axis=-1)
    
    # Return movie feature vector
    movie_features = np.hstack([genre_encoded, title_embeddings]) #, movie_id_embeddings])
    return movie_features

# Prepare data
user_features = preprocess_user_features(data)
movie_features = preprocess_movie_features(data)

# Ensure user_features and movie_features are ready for model input
print("User Features Shape:", user_features.shape)
print("Movie Features Shape:", movie_features.shape)

/Users/mohamedaminemrabet/miniforge3/envs/deep/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


AttributeError: 'list' object has no attribute 'strip'

In [38]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MultiLabelBinarizer
from sentence_transformers import SentenceTransformer
from tqdm import tqdm  # For progress bar

# Load the dataset
data = pd.read_csv("../data/movielens.csv")

# data = df

# Initialize transformers
genre_encoder = MultiLabelBinarizer()
occupation_encoder = OneHotEncoder(sparse_output=False)
scaler = StandardScaler()
title_transformer = SentenceTransformer('all-MiniLM-L6-v2')

genre_map = {
    0: 'Unknown',
    1: 'Action',
    2: 'Adventure',
    3: 'Animation',
    4: 'Children',
    5: 'Comedy',
    6: 'Crime',
    7: 'Documentary',
    8: 'Drama',
    9: 'Fantasy',
    10: 'Film-Noir',
    11: 'Horror',
    12: 'Musical',
    13: 'Mystery',
    14: 'Romance',
    15: 'Sci-Fi',
    16: 'Thriller',
    17: 'War',
    18: 'Western',
    19: 'Other'
}


# Process user features
def preprocess_user_features(data):
    # Initialize tqdm for the processing steps
    with tqdm(total=3, desc="Preprocessing User Features", unit="step") as pbar:
        # Normalize bucketized_user_age
        bucketized_user_age = scaler.fit_transform(data[['bucketized_user_age']])
        pbar.update(1)

        # Encode user_gender as binary (0: False, 1: True)
        user_gender = data['user_gender'].astype(int).values.reshape(-1, 1)
        pbar.update(1)

        # One-hot encode user_occupation_label
        occupation_encoded = occupation_encoder.fit_transform(data[['user_occupation_text']])
        pbar.update(1)

    # Return user feature vector
    user_features = np.hstack([bucketized_user_age, user_gender, occupation_encoded])
    return user_features


# Process movie features
def preprocess_movie_features(data):
    """
    Preprocess movie features for model input.

    Args:
        data (pd.DataFrame): DataFrame containing movie features, including 'movie_genres' and 'movie_title'.
        title_transformer: Sentence transformer model for title embeddings.
        genre_encoder: MultiLabelBinarizer for encoding genres.

    Returns:
        np.ndarray: Preprocessed movie feature vectors.
    """
    # Initialize tqdm for movie genres and titles
    with tqdm(total=2, desc="Preprocessing Movie Features", unit="step") as pbar:
        # Map genres to their string values and one-hot encode them
        if isinstance(data["movie_genres"].iloc[0], str):
            # If it's a string, convert it to a list of integers
            data["movie_genres"] = data["movie_genres"].apply(
                lambda ids: [genre_map[int(id)] for id in ids.strip('[]').split()]
            )
        else:
            # If it's already a list or array of integers, directly map them
            data["movie_genres"] = data["movie_genres"].apply(
                lambda ids: [genre_map[int(id)] for id in ids]
            )
        genre_encoded = genre_encoder.fit_transform(data["movie_genres"])
        pbar.update(1)

        # Encode movie titles with sentence embeddings
        title_embeddings = np.array(title_transformer.encode(data['movie_title'].tolist()))
        pbar.update(1)

    # Return movie feature vector
    movie_features = np.hstack([genre_encoded, title_embeddings])
    return movie_features


# Prepare data
user_features = preprocess_user_features(data)
movie_features = preprocess_movie_features(data)

# Ensure user_features and movie_features are ready for model input
print("User Features Shape:", user_features.shape)
print("Movie Features Shape:", movie_features.shape)


Preprocessing Movie Features: 100%|██████████| 2/2 [02:15<00:00, 67.94s/step]

User Features Shape: (100000, 23)
Movie Features Shape: (100000, 403)


In [46]:
import tensorflow as tf
from tensorflow.keras import layers, models, Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tqdm import tqdm  # For training progress bar

X_users = user_features
X_movies = movie_features
y = data['user_rating'].values  # Labels (e.g., binary rating or numeric)

rating_to_int = {1.: 0, 2.: 0, 3.: 1, 4.: 2, 5.: 2}  # Mapping ratings to integers
y_encoded = np.array([rating_to_int[rating] for rating in y])

# Train/test split
from sklearn.model_selection import train_test_split

X_users_train, X_users_test, X_movies_train, X_movies_test, y_train, y_test = train_test_split(
    X_users, X_movies, y_encoded, test_size=0.2, random_state=42
)

In [47]:
# Convert to TensorFlow tensors
import tensorflow as tf

X_users_train = tf.convert_to_tensor(X_users_train, dtype=tf.float32)
X_movies_train = tf.convert_to_tensor(X_movies_train, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)

X_users_test = tf.convert_to_tensor(X_users_test, dtype=tf.float32)
X_movies_test = tf.convert_to_tensor(X_movies_test, dtype=tf.float32)
y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)

In [48]:
y_train_encoded = to_categorical(y_train, num_classes=3)
y_test_encoded = to_categorical(y_test, num_classes=3)

In [ ]:
# Define the dimensions of the input features
user_feature_dim = user_features.shape[1]  # Adjust based on your data
movie_feature_dim = movie_features.shape[1]  # Adjust based on your data

# User features
user_input = layers.Input(shape=(user_feature_dim,), name="user_input")
user_embedding = layers.Dense(16, activation='relu')(user_input)
user_embedding = layers.BatchNormalization()(user_embedding)  # Batch normalization for user

# Movie features
movie_input = layers.Input(shape=(movie_feature_dim,), name="movie_input")
movie_embedding = layers.Dense(16, activation='relu')(movie_input)
movie_embedding = layers.BatchNormalization()(movie_embedding)  # Batch normalization for movie

## Third model

In [49]:
# Define the dimensions of the input features
user_feature_dim = user_features.shape[1]  # Adjust based on your data
movie_feature_dim = movie_features.shape[1]  # Adjust based on your data

# User input and embedding
user_input = layers.Input(shape=(user_feature_dim,), name="user_input")
user_embedding = layers.Dense(16, activation='relu')(user_input)
user_embedding = layers.BatchNormalization()(user_embedding)

# Movie input and embedding
movie_input = layers.Input(shape=(movie_feature_dim,), name="movie_input")
movie_embedding = layers.Dense(16, activation='relu')(movie_input)
movie_embedding = layers.BatchNormalization()(movie_embedding)

# Combine user and movie embeddings
combined = layers.Concatenate(name="concatenate")([user_embedding, movie_embedding])
# combined = layers.Dense(128, activation='relu')(combined)
# combined = layers.Dropout(0.3)(combined)  # Dropout for regularization
# combined = layers.Dense(64, activation='relu')(combined)

# Output layer with 5 classes
output = layers.Dense(3, activation='softmax', name="output")(combined)

# Final model
model = Model(inputs=[user_input, movie_input], outputs=output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Summary
model.summary()


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ user_input          │ (None, 23)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ movie_input         │ (None, 403)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_12 (Dense)    │ (None, 16)        │        384 │ user_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_13 (Dense)    │ (None, 16)        │      6,464 │ movie_input[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16)        │         64 │ dense_12[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16)        │         64 │ dense_13[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 32)        │          0 │ batch_normalizat… │
│ (Concatenate)       │                   │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 3)         │         99 │ concatenate[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 7,075 (27.64 KB)

 Trainable params: 7,011 (27.39 KB)

 Non-trainable params: 64 (256.00 B)

In [ ]:
# Concatenate user and movie features
merged = layers.concatenate([user_embedding, movie_embedding])

# Final dense layer for recommendation score
output = layers.Dense(1, activation='sigmoid')(merged)

# Build the model
model = models.Model(inputs=[user_input, movie_input], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [43]:
# Model summary
model.summary()

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ user_input          │ (None, 23)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ movie_input         │ (None, 403)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 16)        │        384 │ user_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 16)        │      6,464 │ movie_input[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16)        │         64 │ dense_10[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16)        │         64 │ dense_11[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 32)        │          0 │ batch_normalizat… │
│ (Concatenate)       │                   │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 3)         │         99 │ concatenate[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 7,075 (27.64 KB)

 Trainable params: 7,011 (27.39 KB)

 Non-trainable params: 64 (256.00 B)

In [50]:
# Define callbacks
checkpoint_callback = ModelCheckpoint('../models/best_model.keras', monitor='val_loss', save_best_only=True, mode='min', verbose=1)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)

In [51]:
epochs = 100
batch_size = 64


model.fit(
    [X_users_train, X_movies_train], y_train_encoded, 
    epochs=epochs, 
    batch_size=batch_size,
    validation_data=([X_users_test, X_movies_test], y_test_encoded),  # Use one-hot encoded labels
    callbacks=[checkpoint_callback, early_stopping_callback],
    verbose=1  # Show training progress
)

Epoch 1/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.5018 - loss: 1.0365
Epoch 1: val_loss improved from inf to 0.93050, saving model to ../models/best_model.keras
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 29s 22ms/step - accuracy: 0.5019 - loss: 1.0365 - val_accuracy: 0.5671 - val_loss: 0.9305
Epoch 2/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.5732 - loss: 0.9152
Epoch 2: val_loss improved from 0.93050 to 0.91329, saving model to ../models/best_model.keras
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 26s 21ms/step - accuracy: 0.5732 - loss: 0.9152 - val_accuracy: 0.5752 - val_loss: 0.9133
Epoch 3/100
1248/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.5835 - loss: 0.8981
Epoch 3: val_loss improved from 0.91329 to 0.90685, saving model to ../models/best_model.keras
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 26s 20ms/step - accuracy: 0.5835 - loss: 0.8981 - val_accuracy: 0.5806 - val_loss: 0.9068
Epoch 4/100
1248/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.5822 - loss: 0.8

In [52]:
# Define training data (X_user_train, X_movie_train, y_train should already be defined)
# X_user_train, X_movie_train: Preprocessed features for users and movies
# y_train: Binary labels indicating interactions (1 for liked, 0 for not liked)

# Training with tqdm progress bar for epochs


# Evaluate the model after training
test_loss, test_acc = model.evaluate([X_users_test, X_movies_test], y_test_encoded)  # Test data should be prepared
print(f"Test Loss: {test_loss}, Test Accuracy: {test_acc}")


625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.5835 - loss: 0.8964
Test Loss: 0.8938435316085815, Test Accuracy: 0.5870500206947327


In [53]:
model.save('../models/MT-Model.h5')

## Recommendation

In [54]:
from tensorflow.keras.models import load_model

model = load_model('../models/MT-Model.h5')

In [ ]:
import numpy as np
import tensorflow as tf

def recommend_movies(user_data, movie_data, model, top_n=5):
    """
    Recommend top N movies for a given user based on a multi-class output model.

    Args:
        user_data (pd.DataFrame): DataFrame containing the user features.
        movie_data (pd.DataFrame): DataFrame containing the movie features.
        model (tf.keras.Model): Trained Multi-Tower model.
        top_n (int): Number of top recommendations to return.

    Returns:
        pd.DataFrame: Top N recommended movies with their scores.
    """
    # Preprocess user data into feature vector
    user_features = preprocess_user_features(user_data)
    user_features_tensor = tf.convert_to_tensor(user_features, dtype=tf.float32)

    # Preprocess movie data into feature vectors
    movie_features = preprocess_movie_features(movie_data)
    movie_features_tensor = tf.convert_to_tensor(movie_features, dtype=tf.float32)

    # Replicate user features to match the number of movies
    user_features_tensor = tf.repeat(user_features_tensor, repeats=movie_features_tensor.shape[0], axis=0)

    # Predict scores (multi-class probabilities) for all user-movie pairs
    probabilities = model.predict([user_features_tensor, movie_features_tensor], batch_size=32)

    # Derive scores from probabilities
    scores = np.max(probabilities, axis=1)  # Use the highest probability class as the score

    # Attach scores to the movie_data DataFrame
    movie_data['recommendation_score'] = scores

    # Sort movies by their scores in descending order
    top_movies = movie_data.sort_values('recommendation_score', ascending=False).head(top_n)

    return top_movies[['movie_title', 'recommendation_score']]

# Example usage:
# Assume `user_id` is the user we want to recommend movies for.
df = pd.read_csv('../data/movielens.csv')

df['user_id'] = df['user_id'].apply(trim_col)

user_id = '138'  # Replace with the actual user ID
user_data = df[df['user_id'] == user_id].iloc[0:1].drop(columns=['user_id'])  # Single user's data
movie_data = df[['movie_genres', 'movie_title']]  # All movies

# Get top 5 movie recommendations
top_movies = recommend_movies(user_data, movie_data, model, top_n=5)
print("Top Recommended Movies:")
print(top_movies)

Preprocessing Movie Features:   0%|          | 0/2 [00:00<?, ?step/s]/var/folders/wf/x0xp_htj2zg5fpdgckzglrlr0000gn/T/ipykernel_3011/1016918187.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["movie_genres"] = data["movie_genres"].apply(
Preprocessing Movie Features: 100%|██████████| 2/2 [02:09<00:00, 64.70s/step]


ValueError: Data cardinality is ambiguous. Make sure all arrays contain the same number of samples.'x' sizes: 1, 100000
